## Read CSV

In [14]:
import pandas as pd

In [15]:
# Load the CSV file
df = pd.read_csv('data.csv', index_col = 0)
df.head()

bmi  gender  is_training       name
0  34.207396    Male            1  img_0.bmp
1  26.453720    Male            1  img_1.bmp
2  34.967561  Female            1  img_2.bmp
3  22.044766  Female            1  img_3.bmp
4  37.758789  Female            1  img_4.bmp

In [16]:
import numpy as np

In [17]:
df.shape

(4206, 4)

## Read Image

#### OpenCV

In [18]:
import cv2
import numpy as np

def load_image_cv2(file_path):
    # Load the image file
    img = cv2.imread(file_path)

    # Convert the image to RGB since the later preprocessing code will convert it to BGR
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Resize the image to the size expected by VGGFace (224x224)
    img = cv2.resize(img, (224, 224))
    
    
    return img

In [19]:
import os

# Define the directory that contains your images
image_dir = 'D:\\Uchicago\\2023_Spring\\MSCA-31009\\Project\\Data\\Images\\'

count = 0
for fname in df['name']:
    file_path = os.path.join(image_dir, fname)
    if not os.path.exists(file_path):
        count += 1
print(count)

244


There are 244 missing images

In [20]:
# Initialize an empty list to store the images
images_cv2 = []
# Iterate over the file names
for fname in df['name']:
    file_path = os.path.join(image_dir, fname)
    
    # Check if the file exists
    if os.path.exists(file_path):
        # If the file exists, load the image and append it to the list
        images_cv2.append(load_image_cv2(file_path))
    else:
        images_cv2.append(None)




In [21]:
images_cv2[0]

array([[[176, 194, 216],
        [174, 194, 219],
        [173, 194, 220],
        ...,
        [177, 173, 171],
        [167, 164, 163],
        [164, 162, 163]],

       [[176, 194, 214],
        [174, 194, 218],
        [174, 194, 219],
        ...,
        [178, 175, 173],
        [167, 166, 165],
        [161, 161, 161]],

       [[176, 193, 213],
        [176, 194, 217],
        [175, 193, 218],
        ...,
        [188, 186, 185],
        [179, 178, 178],
        [173, 173, 173]],

       ...,

       [[205,  57,  73],
        [210,  59,  77],
        [214,  60,  80],
        ...,
        [121,  95,  98],
        [123,  97, 101],
        [124,  98, 102]],

       [[204,  61,  75],
        [208,  62,  79],
        [211,  62,  80],
        ...,
        [121,  95,  97],
        [123,  97, 100],
        [125,  98, 101]],

       [[201,  65,  77],
        [204,  65,  79],
        [206,  64,  80],
        ...,
        [121,  95,  96],
        [123,  97,  99],
        [124,  98, 101]]

I first tried this way to preprocess the data but the final prediction of BMI is quite different from that in the paper. Then I found I should use load image from keras and preprocess_input based on the instruction in the github

#### Keras.utils

The VGGFace example used from keras.preprocessing import image but keras.preprocessing API is deprecated in Tensorflow 2.9.1. So we need to use tf.keras.utils instead, to import load_img as follows:

from tensorflow.keras.utils import load_img

In [22]:
import keras.utils as kimage
from keras_vggface.utils import preprocess_input

In [23]:
def load_image_kimage(file_path):
    # Load the image file
    img = kimage.load_img(file_path, target_size=(224, 224))

    x = kimage.img_to_array(img)

    x = preprocess_input(x)
        
    
    return x

In [24]:
# Initialize an empty list to store the images
images_kimage = []
# Iterate over the file names
for fname in df['name']:
    file_path = os.path.join(image_dir, fname)
    
    # Check if the file exists
    if os.path.exists(file_path):
        # If the file exists, load the image and append it to the list
        images_kimage.append(load_image_kimage(file_path))
    else:
        images_kimage.append(None)


In [25]:
images_kimage[0]

array([[[122.406    ,  89.2376   ,  46.813705 ],
        [127.406    ,  89.2376   ,  43.813705 ],
        [127.406    ,  89.2376   ,  43.813705 ],
        ...,
        [ 70.406    ,  60.237602 ,  39.813705 ],
        [ 70.406    ,  60.237602 ,  39.813705 ],
        [ 69.406    ,  57.237602 ,  34.813705 ]],

       [[120.406    ,  89.2376   ,  46.813705 ],
        [125.406    ,  89.2376   ,  44.813705 ],
        [125.406    ,  89.2376   ,  44.813705 ],
        ...,
        [ 76.406    ,  67.2376   ,  43.813705 ],
        [ 76.406    ,  67.2376   ,  43.813705 ],
        [ 66.406    ,  55.237602 ,  30.813705 ]],

       [[120.406    ,  89.2376   ,  46.813705 ],
        [125.406    ,  89.2376   ,  44.813705 ],
        [125.406    ,  89.2376   ,  44.813705 ],
        ...,
        [ 76.406    ,  67.2376   ,  43.813705 ],
        [ 76.406    ,  67.2376   ,  43.813705 ],
        [ 66.406    ,  55.237602 ,  30.813705 ]],

       ...,

       [[-18.594002 , -44.762398 ,  75.813705 ],
        [-1

In [26]:
len(images_kimage)

4206

In [27]:
def load_image_kimage_resnet50(file_path):
    # Load the image file
    img = kimage.load_img(file_path, target_size=(224, 224))

    x = kimage.img_to_array(img)

    x = preprocess_input(x, version = 2)
        
    
    return x

In [28]:
# Initialize an empty list to store the images
images_kimage_resnet50 = []
# Iterate over the file names
for fname in df['name']:
    file_path = os.path.join(image_dir, fname)
    
    # Check if the file exists
    if os.path.exists(file_path):
        # If the file exists, load the image and append it to the list
        images_kimage_resnet50.append(load_image_kimage_resnet50(file_path))
    else:
        images_kimage_resnet50.append(None)

## Combine CSV and Images and Get train test

In [29]:
df['image_kimage'] = images_kimage
df.head(1)

bmi gender  is_training       name  \
0  34.207396   Male            1  img_0.bmp   

                                        image_kimage  
0  [[[122.406, 89.2376, 46.813705], [127.406, 89....

In [30]:
df['image_kimage_resnet50'] = images_kimage_resnet50

In [31]:
df_clean = df.dropna(subset=['image_kimage', 'image_kimage_resnet50'])

In [32]:
df_clean.shape

(3962, 6)

In [33]:
df_clean.head()

bmi  gender  is_training       name  \
0  34.207396    Male            1  img_0.bmp   
1  26.453720    Male            1  img_1.bmp   
2  34.967561  Female            1  img_2.bmp   
3  22.044766  Female            1  img_3.bmp   
6  25.845588  Female            1  img_6.bmp   

                                        image_kimage  \
0  [[[122.406, 89.2376, 46.813705], [127.406, 89....   
1  [[[-93.594, -104.7624, -129.1863], [-93.594, -...   
2  [[[13.405998, 6.237602, 59.813705], [11.405998...   
3  [[[6.405998, 1.2376022, -18.186295], [6.405998...   
6  [[[161.406, 150.23761, 125.813705], [161.406, ...   

                               image_kimage_resnet50  
0  [[[124.5047, 90.1173, 44.9088], [129.5047, 90....  
1  [[[-91.4953, -103.8827, -131.0912], [-91.4953,...  
2  [[[15.5047, 7.117302, 57.9088], [13.5047, 5.11...  
3  [[[8.5047, 2.117302, -20.091202], [8.5047, 2.1...  
6  [[[163.5047, 151.11731, 123.9088], [163.5047, ...

In [39]:
df_clean.image_kimage_resnet50[0].shape

(224, 224, 3)

In [26]:
train_df = df_clean[df_clean['is_training'] == 1] 
train_image = np.array(train_df['image_kimage'].tolist())
train_image_resnet50 = np.array(train_df['image_kimage_resnet50'].tolist())
train_label = np.array(train_df['bmi'])

In [118]:
train_image[:1]

array([[[[122.406    ,  89.2376   ,  46.813705 ],
         [127.406    ,  89.2376   ,  43.813705 ],
         [127.406    ,  89.2376   ,  43.813705 ],
         ...,
         [ 70.406    ,  60.237602 ,  39.813705 ],
         [ 70.406    ,  60.237602 ,  39.813705 ],
         [ 69.406    ,  57.237602 ,  34.813705 ]],

        [[120.406    ,  89.2376   ,  46.813705 ],
         [125.406    ,  89.2376   ,  44.813705 ],
         [125.406    ,  89.2376   ,  44.813705 ],
         ...,
         [ 76.406    ,  67.2376   ,  43.813705 ],
         [ 76.406    ,  67.2376   ,  43.813705 ],
         [ 66.406    ,  55.237602 ,  30.813705 ]],

        [[120.406    ,  89.2376   ,  46.813705 ],
         [125.406    ,  89.2376   ,  44.813705 ],
         [125.406    ,  89.2376   ,  44.813705 ],
         ...,
         [ 76.406    ,  67.2376   ,  43.813705 ],
         [ 76.406    ,  67.2376   ,  43.813705 ],
         [ 66.406    ,  55.237602 ,  30.813705 ]],

        ...,

        [[-18.594002 , -44.762398 ,  7

In [27]:
test_df = df_clean[df_clean['is_training'] == 0] 
test_image = np.array(test_df['image_kimage'].tolist())
test_image_resnet50 = np.array(test_df['image_kimage_resnet50'].tolist())
test_label = np.array(test_df['bmi'])

In [120]:
train_image.shape

(3210, 224, 224, 3)

In [8]:
test_df = df_clean[df_clean['is_training'] == 0] 
test_image = np.array(test_df['image_kimage'].tolist())
test_image_resnet50 = np.array(test_df['image_kimage_resnet50'].tolist())
test_label = np.array(test_df['bmi'])

In [121]:
test_image[:5]

array([[[[ 11.405998  ,  13.237602  ,   2.8137054 ],
         [  7.405998  ,   9.237602  ,  -1.1862946 ],
         [  9.405998  ,  11.237602  ,   0.81370544],
         ...,
         [ 60.406     ,  71.2376    ,  58.813705  ],
         [ 59.406     ,  70.2376    ,  57.813705  ],
         [ 62.406     ,  70.2376    ,  58.813705  ]],

        [[ 12.405998  ,  14.237602  ,   3.8137054 ],
         [  6.405998  ,   8.237602  ,  -2.1862946 ],
         [  8.405998  ,  10.237602  ,  -0.18629456],
         ...,
         [ 56.406     ,  68.2376    ,  58.813705  ],
         [ 58.406     ,  70.2376    ,  60.813705  ],
         [ 65.406     ,  73.2376    ,  63.813705  ]],

        [[  8.405998  ,  10.237602  ,  -0.18629456],
         [  7.405998  ,   9.237602  ,  -1.1862946 ],
         [  8.405998  ,  10.237602  ,  -0.18629456],
         ...,
         [ 60.406     ,  71.2376    ,  58.813705  ],
         [ 58.406     ,  69.2376    ,  56.813705  ],
         [ 62.406     ,  70.2376    ,  58.813705  ]],

In [29]:

# Save train_image array
np.save('train_image.npy', train_image)
np.save('train_image_resnet.npy', train_image_resnet50)

# Save train_label array
np.save('train_label.npy', train_label)


In [30]:
# Save test_image array
np.save('test_image.npy', test_image)
np.save('test_image_resnet.npy', test_image_resnet50)

# Save train_label array
np.save('test_label.npy', test_label)

In [26]:
df_clean.to_csv('df_clean.csv')

#### Different Gender

In [46]:
df_clean.head()

bmi  gender  is_training       name  \
0  34.207396    Male            1  img_0.bmp   
1  26.453720    Male            1  img_1.bmp   
2  34.967561  Female            1  img_2.bmp   
3  22.044766  Female            1  img_3.bmp   
6  25.845588  Female            1  img_6.bmp   

                                        image_kimage  \
0  [[[122.406, 89.2376, 46.813705], [127.406, 89....   
1  [[[-93.594, -104.7624, -129.1863], [-93.594, -...   
2  [[[13.405998, 6.237602, 59.813705], [11.405998...   
3  [[[6.405998, 1.2376022, -18.186295], [6.405998...   
6  [[[161.406, 150.23761, 125.813705], [161.406, ...   

                               image_kimage_resnet50  
0  [[[124.5047, 90.1173, 44.9088], [129.5047, 90....  
1  [[[-91.4953, -103.8827, -131.0912], [-91.4953,...  
2  [[[15.5047, 7.117302, 57.9088], [13.5047, 5.11...  
3  [[[8.5047, 2.117302, -20.091202], [8.5047, 2.1...  
6  [[[163.5047, 151.11731, 123.9088], [163.5047, ...

In [49]:
male_train_df.shape

(1927, 6)

In [50]:
female_train_df.shape

(1283, 6)

In [41]:
train_df = df_clean[df_clean['is_training'] == 1] 
male_train_df = train_df[train_df['gender'] == 'Male']
male_train_image_senet50 = np.array(male_train_df['image_kimage_resnet50'].tolist())
male_train_label = np.array(male_train_df['bmi'])

female_train_df = train_df[train_df['gender'] == 'Female']
female_train_image_senet50 = np.array(female_train_df['image_kimage_resnet50'].tolist())
female_train_label = np.array(female_train_df['bmi'])

In [42]:
test_df = df_clean[df_clean['is_training'] == 0]
male_test_df = test_df[test_df['gender'] == 'Male']
male_test_image_senet50 = np.array(male_test_df['image_kimage_resnet50'].tolist())
male_test_label = np.array(male_test_df['bmi'])

female_test_df = test_df[test_df['gender'] == 'Female']
female_test_image_senet50 = np.array(female_test_df['image_kimage_resnet50'].tolist())
female_test_label = np.array(female_test_df['bmi'])

In [43]:
male_train_image_senet50.shape

(1927, 224, 224, 3)

In [44]:
np.save('male_train_image_senet.npy', male_train_image_senet50)
np.save('male_train_label.npy', male_train_label)
np.save('male_test_image_senet.npy', male_test_image_senet50)
np.save('male_test_label.npy', male_test_label)

In [45]:
np.save('female_train_image_senet.npy', female_train_image_senet50)
np.save('female_train_label.npy', female_train_label)
np.save('female_test_image_senet.npy', female_test_image_senet50)
np.save('female_test_label.npy', female_test_label)

## Test

keras.preprocessing API is deprecated in Tensorflow 2.9.1. Please use tf.keras.utils instead, to import load_img

In [79]:
image_dir = 'D:\\Uchicago\\2023_Spring\\MSCA-31009\\Project\\Data\\Images\\'
test_file_path = os.path.join(image_dir, df.name[0])

In [52]:
test_keras = kimage.load_img(file_path, target_size=(224, 224))
test_keras = kimage.img_to_array(test_keras)
print(test_keras)

[[[176. 194. 216.]
  [173. 194. 221.]
  [173. 194. 221.]
  ...
  [169. 165. 164.]
  [169. 165. 164.]
  [164. 162. 163.]]

 [[176. 194. 214.]
  [174. 194. 219.]
  [174. 194. 219.]
  ...
  [173. 172. 170.]
  [173. 172. 170.]
  [160. 160. 160.]]

 [[176. 194. 214.]
  [174. 194. 219.]
  [174. 194. 219.]
  ...
  [173. 172. 170.]
  [173. 172. 170.]
  [160. 160. 160.]]

 ...

 [[205.  60.  75.]
  [213.  62.  81.]
  [213.  62.  81.]
  ...
  [123.  97. 100.]
  [123.  97. 100.]
  [125.  99. 102.]]

 [[205.  60.  75.]
  [213.  62.  81.]
  [213.  62.  81.]
  ...
  [123.  97. 100.]
  [123.  97. 100.]
  [125.  99. 102.]]

 [[201.  65.  77.]
  [207.  65.  81.]
  [207.  65.  81.]
  ...
  [122.  96.  97.]
  [122.  96.  97.]
  [124.  98. 101.]]]


In [53]:
test_cv2 = cv2.imread(file_path)
test_cv2 = cv2.resize(test_cv2, (224, 224))
test_cv2 = cv2.cvtColor(test_cv2, cv2.COLOR_BGR2RGB)
print(test_cv2)

[[[176 194 216]
  [174 194 219]
  [173 194 220]
  ...
  [177 173 171]
  [167 164 163]
  [164 162 163]]

 [[176 194 214]
  [174 194 218]
  [174 194 219]
  ...
  [178 175 173]
  [167 166 165]
  [161 161 161]]

 [[176 193 213]
  [176 194 217]
  [175 193 218]
  ...
  [188 186 185]
  [179 178 178]
  [173 173 173]]

 ...

 [[205  57  73]
  [210  59  77]
  [214  60  80]
  ...
  [121  95  98]
  [123  97 101]
  [124  98 102]]

 [[204  61  75]
  [208  62  79]
  [211  62  80]
  ...
  [121  95  97]
  [123  97 100]
  [125  98 101]]

 [[201  65  77]
  [204  65  79]
  [206  64  80]
  ...
  [121  95  96]
  [123  97  99]
  [124  98 101]]]


In [78]:
test_pil = Image.open(file_path)
test_pil = test_pil.resize((224, 224))
test_pil = np.array(test_pil)
print(np.array(test_pil))

[[[176 194 216]
  [174 194 219]
  [173 194 221]
  ...
  [177 173 171]
  [165 162 162]
  [164 162 163]]

 [[176 194 215]
  [175 194 218]
  [174 194 220]
  ...
  [177 174 172]
  [165 163 162]
  [158 158 158]]

 [[176 193 213]
  [176 194 217]
  [175 194 219]
  ...
  [189 187 186]
  [178 178 177]
  [173 173 173]]

 ...

 [[205  57  73]
  [210  58  77]
  [215  60  80]
  ...
  [120  95  98]
  [124  98 101]
  [125  98 102]]

 [[204  62  76]
  [208  63  80]
  [211  63  81]
  ...
  [121  95  97]
  [124  98 100]
  [125  99 102]]

 [[201  65  77]
  [205  65  80]
  [207  64  80]
  ...
  [121  95  96]
  [123  97  99]
  [124  98 101]]]


In [77]:
print(test_pil.shape)

(244, 244, 3)


Conclusion:
1. OpenCV (cv2): Linear interpolation (default)
2. PIL (Pillow): Bilinear interpolation
3. Keras: Bilinear interpolation